In [112]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification, make_multilabel_classification


In [113]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleMLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)


## Introdução

Esse notebook tem como intuito destacar os diferentes tipos de tareas de classificação, exemplificando quais funções de perda do pytorch devemos usar em cada caso. Aqui, temos duas tabelas que sumarizam as informações mais abaixo

| Tipo de Classificação | Funções de Perda              | Formato da Label | Formato da Saída do Modelo | Ativação           |
|------------------------|------------------------------|------------------|----------------------------|--------------------|
| Binária                | `BCELoss`, `BCEWithLogitsLoss` | `[B]` ou `[B, 1]` | `[B, 1]`                    | `Sigmoid`          |
| Multi-Classe           | `CrossEntropyLoss`, `NLLLoss` | `[B]`            | `[B, C]`                    | `Softmax`          |
| Multi-Label            | `BCELoss`, `BCEWithLogitsLoss` | `[B, C]`         | `[B, C]`                    | `Sigmoid`          |


| Função de Perda          | Tarefa            | Ativação Interna | Precisa Aplicar Ativação Antes? | Ativação Aplicada (se necessário) |
|--------------------------|-------------------|------------------|----------------------------------|------------------------------------|
| `BCELoss`               | Binária/Multi-Label | Não              | Sim                              | `Sigmoid`                          |
| `BCEWithLogitsLoss`      | Binária/Multi-Label | Sim              | Não                              | Nenhuma                            |
| `CrossEntropyLoss`       | Multi-Classe       | Sim              | Não                              | Nenhuma                            |
| `NLLLoss`                | Multi-Classe       | Não              | Sim                              | `Softmax`                          |


## Classificação Binária

Classificação binária é aquela em que temos DUAS classes, a 0 e a 1. Nesses casos, a saída de nossa rede será tipicamente um número, que indica a probabilidade da imagem pertencer à classe 1.

Um exemplo de classifcação binária e a classifcação de imagens em fotos de gatos, ou fotos de cachorros, em um conjunto de imagens em que todas as fotos tem exatamente um desses animais.

Para a classificação binária, a saída de nossa rede deve conter uma noção probabilística da entrada pertencer à classe 1, ou seja, ela deve ser um número entre 0 e 1. Para isso, tipicamente usamos a *sigmoid* como a ativação de nossa rede.

Podemos usar 2 funções de perda nesse caso:

- A BCELoss;
- A BCEWithLogitsLoss;

> obs: a BCEWithLogitsLoss já faz o calculo da sigmoid internamente, de forma que, ao usá-la, sua rede não deve ter ativação na camada final.

In [114]:

num_samples = 300
num_classes = 2
num_features = 5

X, y = make_classification(
    n_samples=num_samples,
    n_features=num_features,
    n_informative=4,
    n_redundant=0,
    n_classes=num_classes,
    random_state=42
)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float)

dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


#### BCEWithLogitsLoss

Temos dois casos usando a BCEWithLogitsLoss para classificação binária:
- No primeiro, e mais comum, nossas labels e saídas da rede tem o formato: [B, 1]
- No segundo, as labels e saídas tem o formato: [B]

> aqui, B é o batch size



In [115]:
model = SimpleMLP(input_dim=num_features, hidden_dim=16, output_dim=1)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


model.train()
losses = []
epochs = 10
for epoch in range(epochs):
    epoch_loss = 0.0
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()

        batch_y = batch_y.unsqueeze(1) # Adicionando uma dimensão para que a label seja do formato [B, 1]

        outputs = model(batch_x)

        loss = criterion(outputs, batch_y)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    epoch_loss /= len(dataloader)
    losses.append(epoch_loss)
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {epoch_loss:.4f}")

print(f'outputs shape: {outputs.shape} || X example {outputs[0].detach()}')
print(f'y shape: {batch_y.shape} || y example {batch_y[0]}')


Epoch [1/10] - Loss: 0.6474
Epoch [2/10] - Loss: 0.5402
Epoch [3/10] - Loss: 0.4754
Epoch [4/10] - Loss: 0.4203
Epoch [5/10] - Loss: 0.3652
Epoch [6/10] - Loss: 0.3475
Epoch [7/10] - Loss: 0.3085
Epoch [8/10] - Loss: 0.2704
Epoch [9/10] - Loss: 0.2659
Epoch [10/10] - Loss: 0.2340
outputs shape: torch.Size([12, 1]) || X example tensor([-1.0494])
y shape: torch.Size([12, 1]) || y example tensor([0.])


In [116]:
model = SimpleMLP(input_dim=num_features, hidden_dim=16, output_dim=1)

criterion = nn.BCEWithLogitsLoss()  # For multi-class
optimizer = optim.Adam(model.parameters(), lr=0.01)


model.train()
losses = []
epochs = 10
for epoch in range(epochs):
    epoch_loss = 0.0
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()

        batch_y = batch_y

        outputs = model(batch_x)
        outputs = outputs.squeeze(1) # Tirando uma dimensão para que a saída seja do formato [B]
        loss = criterion(outputs, batch_y)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    epoch_loss /= len(dataloader)
    losses.append(epoch_loss)
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {epoch_loss:.4f}")

print(f'outputs shape: {outputs.shape} || X example {outputs[0].detach()}')
print(f'y shape: {batch_y.shape} || y example {batch_y[0]}')



Epoch [1/10] - Loss: 0.6566
Epoch [2/10] - Loss: 0.5551
Epoch [3/10] - Loss: 0.4756
Epoch [4/10] - Loss: 0.4006
Epoch [5/10] - Loss: 0.3618
Epoch [6/10] - Loss: 0.3145
Epoch [7/10] - Loss: 0.2881
Epoch [8/10] - Loss: 0.2648
Epoch [9/10] - Loss: 0.2560
Epoch [10/10] - Loss: 0.2576
outputs shape: torch.Size([12]) || X example 1.2056090831756592
y shape: torch.Size([12]) || y example 1.0


#### BCELoss

Para a classificação binária, podemos também usar a BCELoss, embora o uso da versão com logits seja preferível. A BCELoss funciona da mesma forma que a BCEWithLogitsLoss, com a diferença que aqui não é feito o cálculo da sigmoid.

In [117]:
model = SimpleMLP(input_dim=num_features, hidden_dim=16, output_dim=1)

criterion = nn.BCELoss()  # For multi-class
optimizer = optim.Adam(model.parameters(), lr=0.01)


model.train()
losses = []
epochs = 10
for epoch in range(epochs):
    epoch_loss = 0.0
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()

        batch_y = batch_y.unsqueeze(1) # Adicionando uma dimensão para que o formato da label seja [B, 1]

        outputs = torch.sigmoid(model(batch_x)) # Tirando a sigmoid da saída da rede. Tipicamente isso estaria dentro do modelo
        loss = criterion(outputs, batch_y)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    epoch_loss /= len(dataloader)
    losses.append(epoch_loss)
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {epoch_loss:.4f}")


print(f'outputs shape: {outputs.shape} || X example {outputs[0].detach()}')
print(f'y shape: {batch_y.shape} || y example {batch_y[0]}')

Epoch [1/10] - Loss: 0.6756
Epoch [2/10] - Loss: 0.5994
Epoch [3/10] - Loss: 0.5245
Epoch [4/10] - Loss: 0.4587
Epoch [5/10] - Loss: 0.4209
Epoch [6/10] - Loss: 0.3617
Epoch [7/10] - Loss: 0.3262
Epoch [8/10] - Loss: 0.2997
Epoch [9/10] - Loss: 0.2748
Epoch [10/10] - Loss: 0.2672
outputs shape: torch.Size([12, 1]) || X example tensor([0.2990])
y shape: torch.Size([12, 1]) || y example tensor([1.])


In [118]:
model = SimpleMLP(input_dim=num_features, hidden_dim=16, output_dim=1)

criterion = nn.BCELoss()  # For multi-class
optimizer = optim.Adam(model.parameters(), lr=0.01)


model.train()
losses = []
epochs = 10
for epoch in range(epochs):
    epoch_loss = 0.0
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()

        batch_y = batch_y

        outputs = torch.sigmoid(model(batch_x)) # Tirando a sigmoid da saída da rede. Tipicamente isso estaria dentro do modelo
        outputs = outputs.squeeze()  # Removendo uma dimensão para que o formato da saída seja [B]
        loss = criterion(outputs.squeeze(), batch_y)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    epoch_loss /= len(dataloader)
    losses.append(epoch_loss)
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {epoch_loss:.4f}")

print(f'outputs shape: {outputs.shape} || X example {outputs[0].detach()}')
print(f'y shape: {batch_y.shape} || y example {batch_y[0]}')


Epoch [1/10] - Loss: 0.6812
Epoch [2/10] - Loss: 0.5939
Epoch [3/10] - Loss: 0.5132
Epoch [4/10] - Loss: 0.4413
Epoch [5/10] - Loss: 0.3875
Epoch [6/10] - Loss: 0.3412
Epoch [7/10] - Loss: 0.3078
Epoch [8/10] - Loss: 0.2844
Epoch [9/10] - Loss: 0.2567
Epoch [10/10] - Loss: 0.2313
outputs shape: torch.Size([12]) || X example 0.9987576007843018
y shape: torch.Size([12]) || y example 1.0


## Classificação Multi-Classe

A classificação multi-classe é aquela em que temos múltiplas classes possíveis, mas apenas uma delas está presente em cada dado. Um exemplo desse tipo de classificação seria a classificação de uma imagem entre gatos, cachorros e alpacas, em um dataset em que exatamente um desses animais está presente por foto.

Nesse tipo de tarefas, a saída de nossa rede deve indicar a probabilidade do dado pertencer a cada classe, ou seja, devemos ter um vetor em que cada valor $x_i$ indica a probabilidade de $X$ pertencer à classe $i$. Para manter a noção de probabilidade, devemos respeitar $\sum_{x_i} = 1$, e para isso usamos a função softmax como ativação final de nossa rede.

Aqui, podemos usar 2 funções de perda no pytorch: a CrossEntropyLoss e a NLLLoss

#### CrossEntropyLoss

Usando a CrossEntropyLoss, o target deve ser um vetor de formato [B], em cada cada elemento é um número natural entre 0 e c-1 (c é o número de classes). Se $y_i = 4$, o dado $i$ pertence à classe 4. A saída da rede, em contrapartida, deve ser um vetor one-hot-encoded das labels, ou seja, deve ter o formato [B, C].

A ativação softmax já é feita internamente na função, e logo não deve estar presente em seu modelo.

In [119]:

num_samples = 300
num_classes = 3
num_features = 5

X, y = make_classification(
    n_samples=num_samples,
    n_features=num_features,
    n_informative=4,
    n_redundant=0,
    n_classes=num_classes,
    random_state=42
)


X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long) # Número inteiro

dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [120]:

model = SimpleMLP(input_dim=num_features, hidden_dim=16, output_dim=num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


model.train()
losses = []
epochs = 10
for epoch in range(epochs):
    epoch_loss = 0.0
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()

        batch_y = batch_y

        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    epoch_loss /= len(dataloader)
    losses.append(epoch_loss)
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {epoch_loss:.4f}")

print(f'outputs shape: {outputs.shape} || X example {outputs[0].detach()}')
print(f'y shape: {batch_y.shape} || y example {batch_y[0]}')


Epoch [1/10] - Loss: 1.0291
Epoch [2/10] - Loss: 0.8586
Epoch [3/10] - Loss: 0.7232
Epoch [4/10] - Loss: 0.5967
Epoch [5/10] - Loss: 0.5121
Epoch [6/10] - Loss: 0.4587
Epoch [7/10] - Loss: 0.4301
Epoch [8/10] - Loss: 0.3975
Epoch [9/10] - Loss: 0.3915
Epoch [10/10] - Loss: 0.3610
outputs shape: torch.Size([12, 3]) || X example tensor([-0.9542,  0.9502,  0.5576])
y shape: torch.Size([12]) || y example 2


#### NLLLoss

A Negative Loss Likelihood funciona igual a entropia curzada anterior, com exceção que aqui o softmax não é feito internamente.

É importante notar que, além do softmax, a NLL também requer que tiremos o log.

In [121]:

model = SimpleMLP(input_dim=num_features, hidden_dim=16, output_dim=num_classes)

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


model.train()
losses = []
epochs = 10
for epoch in range(epochs):
    epoch_loss = 0.0
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()

        batch_y = batch_y

        #outputs = torch.log(torch.nn.functional.softmax(model(batch_x)))
        outputs = torch.nn.functional.log_softmax(model(batch_x), dim=1) # Tirando o log da softmax para passar para a NLL
        loss = criterion(outputs, batch_y)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    epoch_loss /= len(dataloader)
    losses.append(epoch_loss)
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {epoch_loss:.4f}")

print(f'outputs shape: {outputs.shape} || X example {outputs[0].detach()}')
print(f'y shape: {batch_y.shape} || y example {batch_y[0]}')


Epoch [1/10] - Loss: 1.0298
Epoch [2/10] - Loss: 0.8680
Epoch [3/10] - Loss: 0.7542
Epoch [4/10] - Loss: 0.6551
Epoch [5/10] - Loss: 0.5735
Epoch [6/10] - Loss: 0.5301
Epoch [7/10] - Loss: 0.4689
Epoch [8/10] - Loss: 0.4267
Epoch [9/10] - Loss: 0.4030
Epoch [10/10] - Loss: 0.3786
outputs shape: torch.Size([12, 3]) || X example tensor([-1.6816, -1.6763, -0.4670])
y shape: torch.Size([12]) || y example 2


## Multi-Label Classification

A classificação multi-label é aquela em que temos múltiplas classes, e várias delas podem estar presentes simultâneamente em um dado. Um exemplo disso é a identificação de objetos em imagens: cada imagem contém vários objetos diferentes que queremos localizar.

Nesses casos, a saída de nossa rede é um vetor te formato [B, C], em que cada valor é um número entre 0 e 1, indicando a probabilidade daquela classe estar presente naquele dado. Tipicamente, nesse caso, usamos a sigmoid como função de ativação.

Como funções de perda, temos as mesmas que as da classificação binária (note que a classificação multi-label se resume a C classificações binárias distintas). Aqui, no entanto, devemos necessariamente ter uma saída de formato [B, C].

In [122]:
# Make a multi-class dataset
num_samples = 300
num_classes = 5
num_features = 5

X, y = make_multilabel_classification(
    n_samples=num_samples,
    n_features=num_features,
    n_classes=num_classes,
    n_labels=2,
    allow_unlabeled=False,
    random_state=42

)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float)  # Must be long for classification

dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [123]:
model = SimpleMLP(input_dim=num_features, hidden_dim=16, output_dim=num_classes)

criterion = nn.BCEWithLogitsLoss()  # For multi-class
optimizer = optim.Adam(model.parameters(), lr=0.01)


model.train()
losses = []
epochs = 10
for epoch in range(epochs):
    epoch_loss = 0.0
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()

        batch_y = batch_y

        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    epoch_loss /= len(dataloader)
    losses.append(epoch_loss)
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {epoch_loss:.4f}")

print(f'outputs shape: {outputs.shape} || X example {outputs[0].detach()}')
print(f'y shape: {batch_y.shape} || y example {batch_y[0]}')


Epoch [1/10] - Loss: 1.0743
Epoch [2/10] - Loss: 0.6407
Epoch [3/10] - Loss: 0.5543
Epoch [4/10] - Loss: 0.5017
Epoch [5/10] - Loss: 0.4698
Epoch [6/10] - Loss: 0.4504
Epoch [7/10] - Loss: 0.4359
Epoch [8/10] - Loss: 0.4287
Epoch [9/10] - Loss: 0.4067
Epoch [10/10] - Loss: 0.3996
outputs shape: torch.Size([12, 5]) || X example tensor([-1.7118,  2.8388,  1.6806,  0.2157, -3.4287])
y shape: torch.Size([12, 5]) || y example tensor([0., 1., 1., 1., 0.])


In [124]:
model = SimpleMLP(input_dim=num_features, hidden_dim=16, output_dim=num_classes)

criterion = nn.BCELoss()  # For multi-class
optimizer = optim.Adam(model.parameters(), lr=0.01)


model.train()
losses = []
epochs = 10
for epoch in range(epochs):
    epoch_loss = 0.0
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()

        batch_y = batch_y

        outputs = torch.sigmoid(model(batch_x))
        loss = criterion(outputs, batch_y)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    epoch_loss /= len(dataloader)
    losses.append(epoch_loss)
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {epoch_loss:.4f}")

print(f'outputs shape: {outputs.shape} || X example {outputs[0].detach()}')
print(f'y shape: {batch_y.shape} || y example {batch_y[0]}')


Epoch [1/10] - Loss: 0.6376
Epoch [2/10] - Loss: 0.4693
Epoch [3/10] - Loss: 0.4228
Epoch [4/10] - Loss: 0.4049
Epoch [5/10] - Loss: 0.3812
Epoch [6/10] - Loss: 0.3794
Epoch [7/10] - Loss: 0.3775
Epoch [8/10] - Loss: 0.3830
Epoch [9/10] - Loss: 0.3703
Epoch [10/10] - Loss: 0.3672
outputs shape: torch.Size([12, 5]) || X example tensor([0.0288, 0.3328, 0.9738, 0.3886, 0.0761])
y shape: torch.Size([12, 5]) || y example tensor([0., 1., 1., 0., 0.])
